In [12]:
import pandas as pd
import nltk
import re
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
# plotting
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report


basic_stopwords = stopwords.words('english')

df = pd.read_csv("dataset2.csv",encoding='latin')
df = df.drop(df.columns[[1,2,3,4]],axis=1)
df = df.rename(columns={df.columns[0]: 'target', df.columns[1]: 'tweet'})



def preprocess(tweet):
    tweet = tweet.lower() 
    tweet = re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ', tweet)
    tweet = nltk.word_tokenize(tweet)
    tweet = [word for word in tweet if word not in basic_stopwords and word.isalnum()]
    return tweet

df.tweet =df.tweet.apply(lambda x:preprocess(x)) 





0          [upset, update, facebook, texting, might, cry,...
1          [dived, many, times, ball, managed, save, 50, ...
2                    [whole, body, feels, itchy, like, fire]
3                                       [behaving, mad, see]
4                                              [whole, crew]
                                 ...                        
1599994                  [woke, school, best, feeling, ever]
1599995     [thewdb, com, cool, hear, old, walt, interviews]
1599996                [ready, mojo, makeover, ask, details]
1599997    [happy, 38th, birthday, boo, alll, time, tupac...
1599998    [happy, charitytuesday, thenspcc, sparkscharit...
Name: tweet, Length: 1599999, dtype: object


In [19]:
X=df.tweet.astype(str)
y=df.target.astype(str)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state =26105111)

vectoriser = TfidfVectorizer(max_features=500000)
vectoriser.fit_transform(X_train)
print('No. of feature_words: ', len(vectoriser.get_feature_names()))


X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

No. of feature_words:  324760


In [23]:
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)
    categories = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1}n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
 

In [24]:
BNBmodel = BernoulliNB()
BNBmodel.fit(X_train, y_train)
model_Evaluate(BNBmodel)
y_pred1 = BNBmodel.predict(X_test)

              precision    recall  f1-score   support

           0       0.77      0.77      0.77     40100
           4       0.77      0.77      0.77     39900

    accuracy                           0.77     80000
   macro avg       0.77      0.77      0.77     80000
weighted avg       0.77      0.77      0.77     80000



In [25]:
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(X_train, y_train)
model_Evaluate(LRmodel)
y_pred3 = LRmodel.predict(X_test)

              precision    recall  f1-score   support

           0       0.79      0.76      0.77     40100
           4       0.77      0.80      0.78     39900

    accuracy                           0.78     80000
   macro avg       0.78      0.78      0.78     80000
weighted avg       0.78      0.78      0.78     80000

